In [36]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn import ensemble
import re

#Import Data
train_dat = pd.read_csv("train.csv", encoding = "ISO-8859-1", low_memory=False)
test_dat = pd.read_csv("test.csv")
train_dat_cl = train_dat.dropna(axis=0, subset=["compliance"])

In [24]:
#Overall Compliance Rate
train_dat["compliance"] = train_dat["compliance"].fillna(-1)
numb_compl = sum(train_dat["compliance"] == 1)
numb_non_compl = sum(train_dat["compliance"] == 0)
numb_not_responsible = sum(train_dat["compliance"] == -1)
compl_rate = numb_compl/(numb_compl + numb_non_compl)
print("Compliance:", numb_compl, "Non-Compliance:", numb_non_compl, "Not Responsible:", numb_not_responsible)
print("Compliance Rate:", compl_rate)

Compliance: 11597 Non-Compliance: 148283 Not Responsible: 90426
Compliance Rate: 0.0725356517388


In [41]:
#Detroit vs Non-Detroit Mailing Address as new feature
train_dat["city"] = [x.lower() for x in train_dat["city"]]
train_dat["inDetroit"] = np.zeros(train_dat.shape[0])
for city in train_dat["city"]:
    match_obj = re.match("det", city)
    if match_obj:
        train_dat["inDetroit"] == 1

In [42]:
# clf = ensemble.RandomForestClassifier()
# clf.fit(train_dat_cl["city"], train_dat_cl["compliance"])
# y_pred = clf.predict(test_dat["city"])
# y_true = test_dat["compliance"]
# metrics.roc_auc_score(y_true, y_pred)